In [2]:
from datetime import datetime,timedelta,date
from time import time
import requests
import pandas as pd
from datetime import date,datetime
import json
from ta.volatility import BollingerBands
from ta import add_all_ta_features


In [3]:
date0,date1 = datetime(2016,1,1) ,datetime.now() - timedelta(days=1)

days = (date1 - date0).days
print(days)

2314


In [4]:
DataFrame = pd.DataFrame(columns = ["time","low","high","open","close","volume","date"])


In [5]:
DataFrame

,time,low,high,open,close,volume,date


In [6]:
dates = []
d0 = date0
d1 = date1
date =[]
for i in range(1,(days//300)+ 1):
    
    d1 = d0 + timedelta(days=300)
    date = [d0.strftime("%Y-%m-%d"),d1.strftime("%Y-%m-%d")]
    dates.append(date)
    d0 = d1

d1 = d0 + timedelta(days= days % 300)
date = [d0.strftime("%Y-%m-%d"),d1.strftime("%Y-%m-%d")]
dates.append(date)


In [7]:

#API_MAIN_LINK
url = "https://api.exchange.coinbase.com"

#Currency name to scrap price 
sym = 'BTC-USD'

#start & ending time of period with size of candle
for i in dates:
    parameters = {
        'start':i[0],
        'end':i[1],
        'granularity':'86400'
    }
    data = requests.get(f"{url}/products/{sym}/candles",params=parameters,headers={'Accepts':'application/json'})
    df = pd.DataFrame(data.json(),columns = ["time","low","high","open","close","volume"])
    df["date"] = pd.to_datetime(df["time"],unit='s')
    DataFrame = DataFrame.append(df)


In [8]:
DataFrame.sort_values(by='date',inplace=True)



In [9]:
DataFrame.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2315 entries, 299 to 0
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   time    2315 non-null   object        
 1   low     2315 non-null   float64       
 2   high    2315 non-null   float64       
 3   open    2315 non-null   float64       
 4   close   2315 non-null   float64       
 5   volume  2315 non-null   float64       
 6   date    2315 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 144.7+ KB


In [10]:
DataFrame.drop_duplicates(inplace=True)


In [11]:
DataFrame.reset_index(inplace=True)

In [12]:
DataFrame.drop('index',axis=1,inplace=True)

In [13]:
z = DataFrame[['date','open','high','close','low','volume','time']]

In [16]:
z

,date,open,high,close,low,volume,time
0,2016-01-02,435.67,437.56,435.40,432.41,3276.709621,1451692800
1,2016-01-03,435.40,435.75,431.91,425.02,3904.335318,1451779200
2,2016-01-04,431.90,435.79,433.85,431.37,5894.445723,1451865600
3,2016-01-05,433.84,435.64,433.34,430.00,5150.109476,1451952000
4,2016-01-06,433.32,433.46,430.87,428.15,5476.959959,1452038400
...,...,...,...,...,...,...,...
2309,2022-04-29,39741.21,39926.80,38593.10,38161.85,14282.259935,1651190400
2310,2022-04-30,38592.27,38795.00,37644.10,37590.00,6931.072001,1651276800
2311,2022-05-01,37640.35,38675.37,38473.05,37401.00,7980.899760,1651363200
2312,2022-05-02,38469.49,39181.27,38510.89,38041.00,17946.983463,1651449600


In [18]:
bb_indicator = BollingerBands(close = DataFrame['close'],window=20,window_dev=2,fillna = True)

In [19]:
z['bb_bbm'] = bb_indicator.bollinger_mavg()
z['bb_bbh'] = bb_indicator.bollinger_hband()
z['bb_bbl'] = bb_indicator.bollinger_lband()

C:\Users\shayanTH\AppData\Local\Temp\ipykernel_16952\3998541396.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  z['bb_bbm'] = bb_indicator.bollinger_mavg()
C:\Users\shayanTH\AppData\Local\Temp\ipykernel_16952\3998541396.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  z['bb_bbh'] = bb_indicator.bollinger_hband()
C:\Users\shayanTH\AppData\Local\Temp\ipykernel_16952\3998541396.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

In [21]:
DataFrame = z

In [22]:
DataFrame.to_csv('Prices.csv')